In [ ]:
import re
from functools import lru_cache

def quaternion_sign(i, j):
    """
    Returns the sign for the quaternion multiplication e_i * e_j,
    based on a hardcoded 4x4 table.
    
          e0   e1   e2   e3  
        -----------------------
    e0 |   1    1    1    1  
    e1 |   1   -1    1   -1  
    e2 |   1   -1   -1    1  
    e3 |   1    1   -1   -1  
    """
    table = [
        [1, 1, 1, 1],
        [1, -1, 1, -1],
        [1, -1, -1, 1],
        [1, 1, -1, -1]
    ]
    
    return table[i][j]

@lru_cache(maxsize=None)
def cayley_dickson_table(n):
    """
    Recursively generates a multiplication table for the Cayley–Dickson algebra
    of dimension 2^n.
    
    Each table entry at position (i, j) is a tuple (s, k) where:
      - s is the sign (+1 or -1) determined by the recursive construction.
      - k is the index of the resulting basis element, computed as i XOR j.
    """
    if n == 2:  # Base case: Quaternion table (4-dimensional)
        dim = 4
        return [[(quaternion_sign(i, j), i ^ j) for j in range(dim)] for i in range(dim)]
    
    # Recursive case: Build table for dimension 2^n from 2^(n-1)
    prev_table = cayley_dickson_table(n - 1)
    prev_dim = 1 << (n - 1)
    dim = 1 << n
    table = [[None] * dim for _ in range(dim)]
    
    # Fill blocks a, b, c, and d in one pass.
    for i in range(prev_dim):
        for j in range(prev_dim):
            base_val = prev_table[i][j]
            table[i][j] = base_val  # Block 'a'
            new_val = (-base_val[0], base_val[1] + prev_dim)
            table[i][j + prev_dim] = new_val  # Block 'b'
            table[i + prev_dim][j] = new_val  # Block 'c'
            table[i + prev_dim][j + prev_dim] = new_val  # Block 'd'
    
    # Adjust signs based on overall and subtable positions.
    for i in range(dim):
        local_row = i % prev_dim
        top = (i < prev_dim)
        for j in range(dim):
            local_col = j % prev_dim
            left = (j < prev_dim)
            s, _ = table[i][j]
            
            # Overall rules.
            if i == 0 or j == 0:
                s = 1
            if i == j:
                s = 1 if i == 0 else -1
            
            # Determine subtable.
            in_a = top and left
            in_b = top and not left
            in_c = not top and left
            in_d = not top and not left
            
            # Subtable main diagonal rule.
            if local_row == local_col:
                if in_a or in_b:
                    if n > 2 and local_row < 4:
                        s = quaternion_sign(local_row, local_col)
                    elif local_row == 0:
                        s = 1
                    else:
                        s = -1
                elif in_c:
                    s = 1
                elif in_d:
                    s = -1
            
            # First column rule in subtable.
            if local_col == 0:
                if in_a or in_b:
                    s = 1
                elif in_c or in_d:
                    s = -1 if local_row == 0 else 1
            
            # First row rule in subtable.
            if local_row == 0:
                if in_a or in_b or (in_c and local_col == 0):
                    s = 1
                elif in_d:
                    s = -1
            
            # Additional adjustments for block 'd'.
            if in_d:
                if local_row == 0 and local_col > 0:
                    s = 1
                if local_col == 0 and local_row > 0:
                    s = -1
            
            table[i][j] = (s, i ^ j)
    
    return table

def multiply_elements(a, b, n):
    """
    Multiplies two hypercomplex elements 'a' and 'b' in a Cayley–Dickson algebra
    of dimension 2^n.
    
    Both 'a' and 'b' are lists of coefficients representing:
        a = a[0]*e0 + a[1]*e1 + ... + a[2^n - 1]*e_{2^n-1}
    
    Multiplication is performed using:
        e_i * e_j = s_{ij} * e_{i XOR j}
    """
    table = cayley_dickson_table(n)
    dim = 1 << n
    result = [0] * dim
    for i in range(dim):
        ai = a[i]
        if ai == 0:
            continue
        for j in range(dim):
            bj = b[j]
            if bj == 0:
                continue
            coeff = ai * bj
            s, k = table[i][j]
            result[k] += coeff * s
    return result

def format_element(elem):
    """
    Returns a formatted string representation of a hypercomplex element given
    its list of coefficients.
    
    For example, [1, 2, -3, 0] becomes "1 + 2e1 - 3e2".
    """
    terms = []
    for i, coeff in enumerate(elem):
        if coeff == 0:
            continue
        if i == 0:
            term = f"{coeff}"
        else:
            if coeff == 1:
                term = f"e{i}"
            elif coeff == -1:
                term = f"-e{i}"
            else:
                term = f"{coeff}e{i}"
        terms.append(term)
    if not terms:
        return "0"
    formatted = terms[0]
    for term in terms[1:]:
        formatted += " - " + term[1:] if term.startswith("-") else " + " + term
    return formatted

def parse_element(s, dim):
    """
    Parses a hypercomplex element provided as a string (e.g., "1 + 2e1 - 3e2" or "e2")
    and returns a list of coefficients of length 'dim' corresponding to:
    
        a = a[0]*e0 + a[1]*e1 + ... + a[dim-1]*e_{dim-1}
    """
    s = s.replace(" ", "")
    if not s:
        raise ValueError("Empty input for hypercomplex element.")
    if s[0] not in "+-":
        s = "+" + s
    tokens = re.findall(r'[+-][^+-]+', s)
    coeffs = [0.0] * dim
    for token in tokens:
        if 'e' in token:
            coeff_str, index_str = token.split('e', 1)
            coeff = 1.0 if coeff_str in ("+", "-") else float(coeff_str)
            index = int(index_str)
        else:
            coeff = float(token)
            index = 0
        if index < 0 or index >= dim:
            raise ValueError(f"Basis index {index} out of range for dimension {dim}.")
        coeffs[index] += coeff
    return coeffs

def main():
    print("Hypercomplex Element Multiplication via Cayley–Dickson Construction")
    
    try:
        n = int(input("Enter n (n=1 for Complex, n=2 for Quaternions, n=3 for Octonions, etc.): "))
    except ValueError:
        print("Error: n must be an integer.")
        return
    
    dim = 1 << n  # 2^n
    print(f"\nThis algebra has dimension {dim} (basis elements e0, e1, ..., e{dim-1}).\n")
    
    try:
        a = parse_element(input("Enter the first element (e.g., '1 + 2e1 - 3e2'):\n"), dim)
    except ValueError as e:
        print("Error parsing first element:", e)
        return
    
    try:
        b = parse_element(input("\nEnter the second element (e.g., '4 - e1 + 0.5e3'):\n"), dim)
    except ValueError as e:
        print("Error parsing second element:", e)
        return
    
    prod = multiply_elements(a, b, n)
    print("\nResult of multiplication:")
    print(f"{format_element(a)} * {format_element(b)} = {format_element(prod)}")
    
    table = cayley_dickson_table(n)
    print(f"\nCayley–Dickson Multiplication Table (n={n}, dimension={dim}):")
    for row in table:
        print("\t".join(f"{'+' if s == 1 else '-'}e{k}" for s, k in row))
    print("-" * 40)

if __name__ == "__main__":
    main()





Hypercomplex Element Multiplication via Cayley–Dickson Construction

This algebra has dimension 4 (basis elements e0, e1, ..., e3).


Result of multiplication:
e1 * e3 = -e2

Cayley–Dickson Multiplication Table (n=2, dimension=4):
+e0	+e1	+e2	+e3
+e1	-e0	+e3	-e2
+e2	-e3	-e0	+e1
+e3	+e2	-e1	-e0
----------------------------------------


In [17]:
import re
import csv
from functools import lru_cache

def quaternion_sign(i, j):
    """Returns the sign for the quaternion multiplication e_i * e_j."""
    table = [
        [1, 1, 1, 1],
        [1, -1, 1, -1],
        [1, -1, -1, 1],
        [1, 1, -1, -1]
    ]
    return table[i][j]

@lru_cache(maxsize=None)
def cayley_dickson_table(n):
    """Generates a multiplication table for the Cayley-Dickson algebra of dimension 2^n."""
    if n == 2:  # Base case: Quaternion table
        dim = 4
        return [[(quaternion_sign(i, j), i ^ j) for j in range(dim)] for i in range(dim)]

    prev_table = cayley_dickson_table(n - 1)
    prev_dim = 1 << (n - 1)
    dim = 1 << n
    table = [[None] * dim for _ in range(dim)]

    for i in range(prev_dim):
        for j in range(prev_dim):
            base_val = prev_table[i][j]
            table[i][j] = base_val  # Block 'a'
            new_val = (-base_val[0], base_val[1] + prev_dim)
            table[i][j + prev_dim] = new_val  # Block 'b'
            table[i + prev_dim][j] = new_val  # Block 'c'
            table[i + prev_dim][j + prev_dim] = new_val  # Block 'd'

    for i in range(dim):
        local_row = i % prev_dim
        top = (i < prev_dim)
        for j in range(dim):
            local_col = j % prev_dim
            left = (j < prev_dim)
            s, _ = table[i][j]
            if i == 0 or j == 0:
                s = 1
            if i == j:
                s = 1 if i == 0 else -1
            in_a = top and left
            in_b = top and not left
            in_c = not top and left
            in_d = not top and not left
            if local_row == local_col:
                if in_a or in_b:
                    if n > 2 and local_row < 4:
                        s = quaternion_sign(local_row, local_col)
                    elif local_row == 0:
                        s = 1
                    else:
                        s = -1
                elif in_c:
                    s = 1
                elif in_d:
                    s = -1
            if local_col == 0:
                if in_a or in_b:
                    s = 1
                elif in_c or in_d:
                    s = -1 if local_row == 0 else 1
            if local_row == 0:
                if in_a or in_b or (in_c and local_col == 0):
                    s = 1
                elif in_d:
                    s = -1
            if in_d:
                if local_row == 0 and local_col > 0:
                    s = 1
                if local_col == 0 and local_row > 0:
                    s = -1
            table[i][j] = (s, i ^ j)

    return table

def multiply_elements(a, b, n):
    """Multiplies two hypercomplex elements 'a' and 'b'."""
    table = cayley_dickson_table(n)
    dim = 1 << n
    result = [0] * dim
    for i in range(dim):
        ai = a[i]
        if ai == 0:
            continue
        for j in range(dim):
            bj = b[j]
            if bj == 0:
                continue
            coeff = ai * bj
            s, k = table[i][j]
            result[k] += coeff * s
    return result

def format_element(elem):
    """Formats a hypercomplex element into a human-readable string."""
    terms = []
    for i, coeff in enumerate(elem):
        if coeff == 0:
            continue
        if i == 0:
            term = f"{coeff}"
        else:
            term = f"{'+' if coeff > 0 else ''}{coeff}e{i}" if coeff != 1 and coeff != -1 else f"{'+' if coeff>0 else ''}e{i}"
            term = term.replace("+-","-")
        terms.append(term)

    if not terms:
        return "0"
    return "".join(terms)

def parse_element(s, dim):
    """Parses a hypercomplex element string into a list of coefficients."""
    s = s.replace(" ", "")
    if not s:
        raise ValueError("Empty input")
    if s[0] not in "+-":
        s = "+" + s
    tokens = re.findall(r'[+-][^+-]+', s)
    coeffs = [0.0] * dim
    for token in tokens:
        if 'e' in token:
            coeff_str, index_str = token.split('e', 1)
            coeff = 1.0 if coeff_str in ("+", "-") else float(coeff_str)
            index = int(index_str)
        else:
            coeff = float(token)
            index = 0
        if index < 0 or index >= dim:
            raise ValueError(f"Index {index} out of range")
        coeffs[index] += coeff
    return coeffs

def save_table_to_csv(table, filename):
    """Saves the Cayley-Dickson multiplication table to a CSV file."""
    dim = len(table)
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        # Write header row (e0, e1, e2, ...)
        header = [f"e{j}" for j in range(dim)]
        writer.writerow([""] + header) # Empty cell in top-left corner

        # Write table data
        for i in range(dim):
            row = [f"e{i}"]  # First element in the row is the basis element
            for sign, index in table[i]:
                row.append(f"{'+' if sign == 1 else '-' }e{index}")
            writer.writerow(row)

def main():
    print("Hypercomplex Element Multiplication & Cayley-Dickson Table Generator")

    try:
        n = int(input("Enter n (n=1 for Complex, n=2 for Quaternions, n=3 for Octonions, etc.): "))
    except ValueError:
        print("Error: n must be an integer.")
        return

    dim = 1 << n
    print(f"\nThis algebra has dimension {dim} (basis elements e0, e1, ..., e{dim-1}).\n")

    # Generate and save the table
    table = cayley_dickson_table(n)
    filename = f"cayley_dickson_table_n{n}.csv"
    save_table_to_csv(table, filename)
    print(f"Cayley-Dickson multiplication table saved to '{filename}'")

    # Optional: Perform multiplication (kept from original script)
    while True:
        try:
            a_str = input("Enter the first element (e.g., '1 + 2e1 - 3e2', or 'q' to quit):\n")
            if a_str.lower() == 'q':
                break
            a = parse_element(a_str, dim)

            b_str = input("\nEnter the second element (e.g., '4 - e1 + 0.5e3'):\n")
            b = parse_element(b_str, dim)

            prod = multiply_elements(a, b, n)
            print("\nResult of multiplication:")
            print(f"{format_element(a)} * {format_element(b)} = {format_element(prod)}")

        except ValueError as e:
            print("Error:", e)

if __name__ == "__main__":
    main()

Hypercomplex Element Multiplication & Cayley-Dickson Table Generator

This algebra has dimension 2048 (basis elements e0, e1, ..., e2047).

Cayley-Dickson multiplication table saved to 'cayley_dickson_table_n11.csv'


In [ ]:
import re
from functools import lru_cache

def quaternion_sign(i, j):
    """
    Returns the sign for the quaternion multiplication e_i * e_j,
    based on a hardcoded 4x4 table.
    
      e0 e1 e2 e3  
    e0  1  1  1  1  
    e1  1 -1  1 -1  
    e2  1 -1 -1  1  
    e3  1  1 -1 -1  
    """
    table = [
        [1, 1, 1, 1],
        [1, -1, 1, -1],
        [1, -1, -1, 1],
        [1, 1, -1, -1]
    ]
    return table[i][j]

@lru_cache(maxsize=None)
def cayley_dickson_table(n):
    """
    Recursively generates a multiplication table for the Cayley–Dickson algebra
    of dimension d = 2^n.
    
    Each table entry at position (i, j) is a tuple (s, k) where:
      - s is the sign (+1 or -1) determined by the recursive construction.
      - k is the index of the resulting basis element, computed as i XOR j.
    """
    if n == 2:  # Base case: Quaternion table (4-dimensional)
        dim = 4
        return [[(quaternion_sign(i, j), i ^ j) for j in range(dim)] for i in range(dim)]
    
    # Recursive case: Build table for dimension 2^n from 2^(n-1)
    prev_table = cayley_dickson_table(n - 1)
    prev_dim = 1 << (n - 1)
    dim = 1 << n
    table = [[None] * dim for _ in range(dim)]
    
    # Step 1: Fill in subtable 'a' (upper left block)
    for i in range(prev_dim):
        for j in range(prev_dim):
            table[i][j] = prev_table[i][j]
    
    # Step 2: Fill in subtables 'b', 'c', and 'd' (negated copies with index shift)
    for i in range(prev_dim):
        for j in range(prev_dim):
            sign, index = prev_table[i][j]
            table[i][j + prev_dim] = (-sign, index + prev_dim)  # Block 'b'
            table[i + prev_dim][j] = (-sign, index + prev_dim)  # Block 'c'
            table[i + prev_dim][j + prev_dim] = (-sign, index + prev_dim)  # Block 'd'
    
    # Step 3: Adjust signs based on overall and subtable positions.
    for i in range(dim):
        for j in range(dim):
            sign, _ = table[i][j]
            local_row = i % prev_dim
            local_col = j % prev_dim
            
            # Overall rules.
            if i == 0 or j == 0:
                sign = 1
            if i == j:
                sign = 1 if i == 0 else -1
            
            # Determine subtable.
            in_a = (i < prev_dim and j < prev_dim)
            in_b = (i < prev_dim and j >= prev_dim)
            in_c = (i >= prev_dim and j < prev_dim)
            in_d = (i >= prev_dim and j >= prev_dim)
            
            # Subtable main diagonal rule.
            if local_row == local_col:
                if in_a or in_b:
                    if n > 2 and local_row < 4 and local_col < 4:
                        sign = quaternion_sign(local_row, local_col)
                    elif local_row == 0:
                        sign = 1
                    else:
                        sign = -1
                elif in_c:
                    sign = 1
                elif in_d:
                    sign = -1
            
            # First column rule in subtable.
            if local_col == 0:
                if in_a or in_b:
                    sign = 1
                elif in_c:
                    sign = -1 if local_row == 0 else 1
                elif in_d:
                    sign = -1 if local_row == 0 else 1
            
            # First row rule in subtable.
            if local_row == 0:
                if in_a or in_b or (in_c and local_col == 0):
                    sign = 1
                elif in_d:
                    sign = -1
            if in_d and local_row == 0 and local_col > 0:
                sign = 1
            if in_d and local_col == 0 and local_row > 0:
                sign = -1
            
            table[i][j] = (sign, i ^ j)
    
    return table

def multiply_elements(a, b, n):
    """
    Multiplies two hypercomplex elements 'a' and 'b' in a Cayley–Dickson algebra
    of dimension 2^n.
    
    Both 'a' and 'b' are lists of real coefficients representing:
        a = a[0]*e0 + a[1]*e1 + ... + a[2^n - 1]*e_{2^n-1}
    
    The multiplication is done using:
        e_i * e_j = s_{ij} * e_{i XOR j}
    """
    table = cayley_dickson_table(n)
    dim = 1 << n
    result = [0] * dim
    for i in range(dim):
        if a[i] == 0:
            continue
        for j in range(dim):
            if b[j] == 0:
                continue
            coeff = a[i] * b[j]
            s, k = table[i][j]
            result[k] += coeff * s
    return result

def format_element(elem):
    """
    Returns a formatted string representation of a hypercomplex element given
    its list of coefficients.
    
    For example, [1, 2, -3, 0] becomes "1 + 2e1 - 3e2".
    """
    terms = []
    for i, coeff in enumerate(elem):
        if coeff == 0:
            continue
        if i == 0:
            term = f"{coeff}"
        else:
            if coeff == 1:
                term = f"e{i}"
            elif coeff == -1:
                term = f"-e{i}"
            else:
                term = f"{coeff}e{i}"
        terms.append(term)
    if not terms:
        return "0"
    result = terms[0]
    for term in terms[1:]:
        result += " - " + term[1:] if term.startswith("-") else " + " + term
    return result

def parse_element(s, dim):
    """
    Parses a hypercomplex element provided as a string (e.g., "1 + 2e1 - 3e2" or "e2")
    and returns a list of coefficients of length 'dim' corresponding to:
    
        a = a[0]*e0 + a[1]*e1 + ... + a[dim-1]*e_{dim-1}
    """
    s = s.replace(" ", "")
    if not s:
        raise ValueError("Empty input for hypercomplex element.")
    if s[0] not in "+-":
        s = "+" + s
    tokens = re.findall(r'[+-][^+-]+', s)
    coeffs = [0.0] * dim
    for token in tokens:
        if 'e' in token:
            parts = token.split('e', 1)
            coeff_str, index_str = parts[0], parts[1]
            if coeff_str in ("+", "-"):
                coeff = 1.0 if coeff_str == "+" else -1.0
            else:
                try:
                    coeff = float(coeff_str)
                except ValueError:
                    raise ValueError(f"Invalid coefficient in token: {token}")
            try:
                index = int(index_str)
            except ValueError:
                raise ValueError(f"Invalid basis index in token: {token}")
        else:
            try:
                coeff = float(token)
            except ValueError:
                raise ValueError(f"Invalid scalar term: {token}")
            index = 0
        if index < 0 or index >= dim:
            raise ValueError(f"Basis index {index} out of range for dimension {dim}.")
        coeffs[index] += coeff
    return coeffs

# Main program using user inputs:
if __name__ == "__main__":
    print("Hypercomplex Element Multiplication via Cayley–Dickson Construction")
    
    #n_input = input("Enter n (n=1 for Complex, n=2 for Quaternions, n=3 for Octonions, etc.): ")
    n=12
    a_str="1+e4+e6+e7"
    b_str="4+e5+e6+e8"
    try:
        n = int(n_input)
    except ValueError:
        print("Error: n must be an integer.")
        exit(1)
    
    dim = 1 << n  # 2^n
    print(f"\nThis algebra has dimension {dim} (basis elements e0, e1, ..., e{dim-1}).\n")
    
    #a_str = input("Enter the first element (e.g., '1 + 2e1 - 3e2'):\n")
    try:
        a = parse_element(a_str, dim)
    except ValueError as e:
        print("Error parsing first element:", e)
        exit(1)
    
    #b_str = input("\nEnter the second element (e.g., '4 - e1 + 0.5e3'):\n")
    try:
        b = parse_element(b_str, dim)
    except ValueError as e:
        print("Error parsing second element:", e)
        exit(1)
    
    prod = multiply_elements(a, b, n)
    print("\nResult of multiplication:")
    print(f"{a_str} * {b_str} = {format_element(prod)}")
    
    table = cayley_dickson_table(n)
    print(f"Cayley–Dickson Multiplication Table (n={n}, dimension={2**n}):")
    for row in table:
        print("\t".join(f"{'+' if s == 1 else '-'}e{k}" for s, k in row))
    print("-" * 40)


In [ ]:
import re
from functools import lru_cache

def quaternion_sign(i, j):
    """
    Returns the sign for the quaternion multiplication e_i * e_j,
    based on a hardcoded 4x4 table.
    
          e0   e1   e2   e3  
        -----------------------
    e0 |   1    1    1    1  
    e1 |   1   -1    1   -1  
    e2 |   1   -1   -1    1  
    e3 |   1    1   -1   -1  
    """
    table = [
        [1, 1, 1, 1],
        [1, -1, 1, -1],
        [1, -1, -1, 1],
        [1, 1, -1, -1]
    ]
    return table[i][j]

@lru_cache(maxsize=None)
def cayley_dickson_table(n):
    """
    Recursively generates a multiplication table for the Cayley–Dickson algebra
    of dimension 2^n.
    
    Each table entry at position (i, j) is a tuple (s, k) where:
      - s is the sign (+1 or -1) determined by the recursive construction.
      - k is the index of the resulting basis element, computed as i XOR j.
    """
    if n == 2:  # Base case: Quaternion table (4-dimensional)
        dim = 4
        return [[(quaternion_sign(i, j), i ^ j) for j in range(dim)] for i in range(dim)]
    
    # Recursive case: Build table for dimension 2^n from 2^(n-1)
    prev_table = cayley_dickson_table(n - 1)
    prev_dim = 1 << (n - 1)
    dim = 1 << n
    table = [[None] * dim for _ in range(dim)]
    
    # Fill blocks a, b, c, and d in one pass.
    for i in range(prev_dim):
        for j in range(prev_dim):
            base_val = prev_table[i][j]
            table[i][j] = base_val  # Block 'a'
            new_val = (-base_val[0], base_val[1] + prev_dim)
            table[i][j + prev_dim] = new_val  # Block 'b'
            table[i + prev_dim][j] = new_val  # Block 'c'
            table[i + prev_dim][j + prev_dim] = new_val  # Block 'd'
    
    # Adjust signs based on overall and subtable positions.
    for i in range(dim):
        local_row = i % prev_dim
        top = (i < prev_dim)
        for j in range(dim):
            local_col = j % prev_dim
            left = (j < prev_dim)
            s, _ = table[i][j]
            
            # Overall rules.
            if i == 0 or j == 0:
                s = 1
            if i == j:
                s = 1 if i == 0 else -1
            
            # Determine subtable.
            in_a = top and left
            in_b = top and not left
            in_c = not top and left
            in_d = not top and not left
            
            # Subtable main diagonal rule.
            if local_row == local_col:
                if in_a or in_b:
                    if n > 2 and local_row < 4:
                        s = quaternion_sign(local_row, local_col)
                    elif local_row == 0:
                        s = 1
                    else:
                        s = -1
                elif in_c:
                    s = 1
                elif in_d:
                    s = -1
            
            # First column rule in subtable.
            if local_col == 0:
                if in_a or in_b:
                    s = 1
                elif in_c or in_d:
                    s = -1 if local_row == 0 else 1
            
            # First row rule in subtable.
            if local_row == 0:
                if in_a or in_b or (in_c and local_col == 0):
                    s = 1
                elif in_d:
                    s = -1
            
            # Additional adjustments for block 'd'.
            if in_d:
                if local_row == 0 and local_col > 0:
                    s = 1
                if local_col == 0 and local_row > 0:
                    s = -1
            
            table[i][j] = (s, i ^ j)
    
    return table

def multiply_elements(a, b, n):
    """
    Multiplies two hypercomplex elements 'a' and 'b' in a Cayley–Dickson algebra
    of dimension 2^n.
    
    Both 'a' and 'b' are lists of coefficients representing:
        a = a[0]*e0 + a[1]*e1 + ... + a[2^n - 1]*e_{2^n-1}
    
    Multiplication is performed using:
        e_i * e_j = s_{ij} * e_{i XOR j}
    """
    table = cayley_dickson_table(n)
    dim = 1 << n
    result = [0] * dim
    for i in range(dim):
        ai = a[i]
        if ai == 0:
            continue
        for j in range(dim):
            bj = b[j]
            if bj == 0:
                continue
            coeff = ai * bj
            s, k = table[i][j]
            result[k] += coeff * s
    return result

def format_element(elem):
    """
    Returns a formatted string representation of a hypercomplex element given
    its list of coefficients.
    
    For example, [1, 2, -3, 0] becomes "1 + 2e1 - 3e2".
    """
    terms = []
    for i, coeff in enumerate(elem):
        if coeff == 0:
            continue
        if i == 0:
            term = f"{coeff}"
        else:
            if coeff == 1:
                term = f"e{i}"
            elif coeff == -1:
                term = f"-e{i}"
            else:
                term = f"{coeff}e{i}"
        terms.append(term)
    if not terms:
        return "0"
    formatted = terms[0]
    for term in terms[1:]:
        formatted += " - " + term[1:] if term.startswith("-") else " + " + term
    return formatted

def parse_element(s, dim):
    """
    Parses a hypercomplex element provided as a string (e.g., "1 + 2e1 - 3e2" or "e2")
    and returns a list of coefficients of length 'dim' corresponding to:
    
        a = a[0]*e0 + a[1]*e1 + ... + a[dim-1]*e_{dim-1}
    """
    s = s.replace(" ", "")
    if not s:
        raise ValueError("Empty input for hypercomplex element.")
    if s[0] not in "+-":
        s = "+" + s
    tokens = re.findall(r'[+-][^+-]+', s)
    coeffs = [0.0] * dim
    for token in tokens:
        if 'e' in token:
            coeff_str, index_str = token.split('e', 1)
            coeff = 1.0 if coeff_str in ("+", "-") else float(coeff_str)
            index = int(index_str)
        else:
            coeff = float(token)
            index = 0
        if index < 0 or index >= dim:
            raise ValueError(f"Basis index {index} out of range for dimension {dim}.")
        coeffs[index] += coeff
    return coeffs

def main():
    print("Hypercomplex Element Multiplication via Cayley–Dickson Construction")
    
    try:
        n = int(input("Enter n (n=1 for Complex, n=2 for Quaternions, n=3 for Octonions, etc.): "))
    except ValueError:
        print("Error: n must be an integer.")
        return
    
    dim = 1 << n  # 2^n
    print(f"\nThis algebra has dimension {dim} (basis elements e0, e1, ..., e{dim-1}).\n")
    
    try:
        a = parse_element(input("Enter the first element (e.g., '1 + 2e1 - 3e2'):\n"), dim)
    except ValueError as e:
        print("Error parsing first element:", e)
        return
    
    try:
        b = parse_element(input("\nEnter the second element (e.g., '4 - e1 + 0.5e3'):\n"), dim)
    except ValueError as e:
        print("Error parsing second element:", e)
        return
    
    prod = multiply_elements(a, b, n)
    print("\nResult of multiplication:")
    print(f"{format_element(a)} * {format_element(b)} = {format_element(prod)}")
    
    table = cayley_dickson_table(n)
    print(f"\nCayley–Dickson Multiplication Table (n={n}, dimension={dim}):")
    for row in table:
        print("\t".join(f"{'+' if s == 1 else '-'}e{k}" for s, k in row))
    print("-" * 40)

if __name__ == "__main__":
    main()


if __name__ == "__main__":
    main()


In [ ]:
def quaternion_sign(i, j):
    """Returns the sign from the hardcoded quaternion table."""
    table = [
        [1, 1, 1, 1],
        [1, -1, 1, -1],
        [1, -1, -1, 1],
        [1, 1, -1, -1]
    ]
    return table[i][j]

def cayley_dickson_table(n):
    """Generates the Cayley-Dickson table iteratively."""
    if n == 2:  # Base case: Quaternion table
        dim = 4
        table = [[(0) for _ in range(dim)] for _ in range(dim)]
        for i in range(dim):
          for j in range(dim):
            table[i][j] = (quaternion_sign(i,j), i^j)
        return table

    # Build from the previous dimension
    prev_table = cayley_dickson_table(n - 1)
    prev_dim = 1 << (n - 1)
    dim = 1 << n
    table = [[(0) for _ in range(dim)] for _ in range(dim)]

    # Step 1: Fill in subtable 'a' (copy previous table)
    for i in range(prev_dim):
        for j in range(prev_dim):
            table[i][j] = prev_table[i][j]

    # Step 2: Fill in 'b', 'c', 'd' (negated 'a')
    for i in range(prev_dim):
        for j in range(prev_dim):
            sign, index = prev_table[i][j]
            table[i][j + prev_dim] = (-sign, index + prev_dim)  # 'b'
            table[i + prev_dim][j] = (-sign, index + prev_dim)  # 'c'
            table[i + prev_dim][j + prev_dim] = (-sign, index + prev_dim)  # 'd'

    # Step 3: Adjust signs based on position
    for i in range(dim):
        for j in range(dim):
            message=""
            sign, index = table[i][j]
            local_row = i % prev_dim
            local_col = j % prev_dim
            # Rule 1: Overall First Row/Column
            if i == 0 or j == 0:
                sign = 1

            # Rule 2: Overall Main Diagonal
            if i == j:
                sign = 1 if i == 0 else -1

            # Determine subtable
            in_a = i < prev_dim and j < prev_dim
            in_b = i < prev_dim and j >= prev_dim
            in_c = i >= prev_dim and j < prev_dim
            in_d = i >= prev_dim and j >= prev_dim

            # Rule 3: Subtable Diagonals
            if local_row == local_col:
                if in_a or in_b:
                    if n > 2 and local_row < 4 and local_col < 4: # use quaternion table for a and b diagonals when n >2
                      sign = quaternion_sign(local_row,local_col)
                    elif local_row == 0:
                        sign = 1
                    else:
                        sign = -1
                elif in_c:
                    sign = 1
                elif in_d:
                    sign = -1

            # Rule 4: Subtable First Columns
            if local_col == 0:
                if in_a or in_b:
                    sign = 1
                elif in_c:
                    sign = -1 if local_row == 0 else 1
                elif in_d:
                    sign = -1 if local_row == 0 else 1

            # Rule 5: Subtable First Rows
            if local_row == 0 :

                if in_a or in_b or (in_c and local_col==0):
                    sign = 1
                elif in_d:
                    sign = -1
            if(in_d and local_row==0 and local_col>0):
              sign=1
              
            if(in_d and local_col==0 and local_row>0):
              sign=-1
                
            #print(i,j,local_col,local_row,sign,message)
            table[i][j] = (sign, i ^ j) #update index

    return table

# Example Usage and Testing:
for n in range(2, 5):  # Start from quaternions (n=2)
    table = cayley_dickson_table(n)
    print(f"Cayley-Dickson Multiplication Table (n={n}, dimension={2**n}):")
    for row in table:
        for sign_val, index in row:
            print(f"{'+' if sign_val == 1 else '-' }e{index}", end="\t")
        print()
    print("-" * 40)

In [ ]:
def quaternion_sign(i, j):
    """
    Returns the sign for the quaternion multiplication e_i * e_j,
    based on a hardcoded 4x4 table. For quaternions:
    
      e0 e1 e2 e3  
    e0  1  1  1  1  
    e1  1 -1  1 -1  
    e2  1 -1 -1  1  
    e3  1  1 -1 -1  

    This is used as the base rule in the Cayley–Dickson construction.
    """
    table = [
        [1, 1, 1, 1],
        [1, -1, 1, -1],
        [1, -1, -1, 1],
        [1, 1, -1, -1]
    ]
    return table[i][j]

def cayley_dickson_table(n):
    """
    Recursively generates a multiplication table for the Cayley–Dickson algebra
    of dimension d = 2^n.
    
    Each table entry at position (i, j) is a tuple (s, k) where:
      - s is the sign (+1 or -1) determined by the recursive construction.
      - k is the index of the resulting basis element, computed as i XOR j.
      
    Base case:
      For n == 2, we generate the quaternion multiplication table.
      
    Recursive case:
      For n > 2, we build the table from the previous dimension (2^(n-1))
      by arranging four blocks (a, b, c, d) and then applying correction
      rules to adjust the signs based on the position in the overall table and
      within each subblock.
    """
    if n == 2:  # Base case: Quaternion table (4-dimensional)
        dim = 4
        table = [[None for _ in range(dim)] for _ in range(dim)]
        for i in range(dim):
            for j in range(dim):
                table[i][j] = (quaternion_sign(i, j), i ^ j)
        return table

    # Build from the previous dimension (recursive doubling)
    prev_table = cayley_dickson_table(n - 1)
    prev_dim = 1 << (n - 1)
    dim = 1 << n  # 2^n
    table = [[None for _ in range(dim)] for _ in range(dim)]

    # Step 1: Fill in subtable 'a' (the upper left block): copy previous table
    for i in range(prev_dim):
        for j in range(prev_dim):
            table[i][j] = prev_table[i][j]

    # Step 2: Fill in subtables 'b', 'c', and 'd' by shifting indices and negating signs
    for i in range(prev_dim):
        for j in range(prev_dim):
            sign, index = prev_table[i][j]
            table[i][j + prev_dim] = (-sign, index + prev_dim)  # Block 'b'
            table[i + prev_dim][j] = (-sign, index + prev_dim)  # Block 'c'
            table[i + prev_dim][j + prev_dim] = (-sign, index + prev_dim)  # Block 'd'

    # Step 3: Adjust signs according to several rules based on overall and subtable positions.
    for i in range(dim):
        for j in range(dim):
            sign, _ = table[i][j]
            local_row = i % prev_dim  # Position within a subtable row
            local_col = j % prev_dim  # Position within a subtable column
            
            # Rule 1: First overall row or column always yield a positive sign.
            if i == 0 or j == 0:
                sign = 1

            # Rule 2: Main diagonal: 1 at (0,0) then -1 for others.
            if i == j:
                sign = 1 if i == 0 else -1

            # Determine in which subtable the (i, j) entry lies.
            in_a = (i < prev_dim and j < prev_dim)
            in_b = (i < prev_dim and j >= prev_dim)
            in_c = (i >= prev_dim and j < prev_dim)
            in_d = (i >= prev_dim and j >= prev_dim)

            # Rule 3: On a subtable’s main diagonal
            if local_row == local_col:
                if in_a or in_b:
                    # For higher n, optionally use the quaternion table for the initial indices
                    if n > 2 and local_row < 4 and local_col < 4:
                        sign = quaternion_sign(local_row, local_col)
                    elif local_row == 0:
                        sign = 1
                    else:
                        sign = -1
                elif in_c:
                    sign = 1
                elif in_d:
                    sign = -1

            # Rule 4: First column of a subtable.
            if local_col == 0:
                if in_a or in_b:
                    sign = 1
                elif in_c:
                    sign = -1 if local_row == 0 else 1
                elif in_d:
                    sign = -1 if local_row == 0 else 1

            # Rule 5: First row of a subtable.
            if local_row == 0:
                if in_a or in_b or (in_c and local_col == 0):
                    sign = 1
                elif in_d:
                    sign = -1
            if in_d and local_row == 0 and local_col > 0:
                sign = 1
            if in_d and local_col == 0 and local_row > 0:
                sign = -1

            table[i][j] = (sign, i ^ j)  # Ensure the index remains i XOR j

    return table

def multiply_elements(a, b, n):
    """
    Multiplies two elements 'a' and 'b' in a Cayley–Dickson algebra of dimension 2^n.
    
    Both 'a' and 'b' are expected to be lists (or vectors) of real coefficients 
    with length 2^n representing:
    
        a = a[0]*e0 + a[1]*e1 + ... + a[2^n - 1]*e_{2^n-1}
    
    The multiplication is performed using the Cayley–Dickson table:
    
        e_i * e_j = s_{ij} * e_{i XOR j}
    
    where s_{ij} is obtained from the Cayley–Dickson table.
    
    Returns the resulting element as a list of coefficients.
    """
    table = cayley_dickson_table(n)
    dim = 1 << n
    # Initialize the result vector with zeros.
    result = [0] * dim
    # Iterate over all pairs of basis indices.
    for i in range(dim):
        for j in range(dim):
            # Only consider nonzero contributions.
            if a[i] == 0 or b[j] == 0:
                continue
            coeff = a[i] * b[j]
            s, k = table[i][j]
            result[k] += coeff * s
    return result

def format_element(elem):
    """
    Returns a human–readable string for an element given by its list of coefficients.
    
    For example, an element [1, 2, -3, 0] (in the quaternion algebra) is returned as:
        "1 + 2e1 - 3e2"
    
    The basis element e0 (the multiplicative identity) is printed as a constant.
    """
    terms = []
    for i, coeff in enumerate(elem):
        if coeff == 0:
            continue  # Skip zero coefficients.
        # For the scalar part e0, do not print the basis symbol.
        if i == 0:
            term = f"{coeff}"
        else:
            # Format the term: omit the coefficient when it is 1 or -1 (but keep the sign).
            if coeff == 1:
                term = f"e{i}"
            elif coeff == -1:
                term = f"-e{i}"
            else:
                term = f"{coeff}e{i}"
        terms.append(term)
    if not terms:
        return "0"
    # Combine the terms with appropriate signs.
    result = terms[0]
    for term in terms[1:]:
        if term.startswith("-"):
            result += " - " + term[1:]
        else:
            result += " + " + term
    return result

# Example Usage:
if __name__ == "__main__":
    # Choose the dimension parameter (n=2 gives quaternions, n=3 gives octonions, etc.)
    n = 2  # working with quaternions (4-dimensional)
    
    # Represent two elements. For example:
    #   a = 1 + 2e1 - 3e2  is represented as [1, 2, -3, 0]
    #   b = 4 + 0e1 + 1e2 - 2e3 is represented as [4, 0, 1, -2]
    a = [1, 0, 1, 0]
    b = [0, 0, 0, 1]
    
    # Multiply the elements using our Cayley–Dickson multiplication routine.
    prod = multiply_elements(a, b, n)
    
    # Print the factors and the result in a human–readable format.
    print("Element a: ", format_element(a))
    print("Element b: ", format_element(b))
    print("Product  : ", format_element(prod))


In [ ]:
def quaternion_sign(i, j):
    """
    Returns the sign for the quaternion multiplication e_i * e_j,
    based on a hardcoded 4x4 table. For quaternions:
    
      e0 e1 e2 e3  
    e0  1  1  1  1  
    e1  1 -1  1 -1  
    e2  1 -1 -1  1  
    e3  1  1 -1 -1  

    This is used as the base rule in the Cayley–Dickson construction.
    """
    table = [
        [1, 1, 1, 1],
        [1, -1, 1, -1],
        [1, -1, -1, 1],
        [1, 1, -1, -1]
    ]
    return table[i][j]

def cayley_dickson_table(n):
    """
    Recursively generates a multiplication table for the Cayley–Dickson algebra
    of dimension d = 2^n.
    
    Each table entry at position (i, j) is a tuple (s, k) where:
      - s is the sign (+1 or -1) determined by the recursive construction.
      - k is the index of the resulting basis element, computed as i XOR j.
      
    Base case:
      For n == 2, we generate the quaternion multiplication table.
      
    Recursive case:
      For n > 2, we build the table from the previous dimension (2^(n-1))
      by arranging four blocks (a, b, c, d) and then applying correction
      rules to adjust the signs based on the position in the overall table and
      within each subblock.
    """
    if n == 2:  # Base case: Quaternion table (4-dimensional)
        dim = 4
        table = [[None for _ in range(dim)] for _ in range(dim)]
        for i in range(dim):
            for j in range(dim):
                table[i][j] = (quaternion_sign(i, j), i ^ j)
        return table

    # Build from the previous dimension (recursive doubling)
    prev_table = cayley_dickson_table(n - 1)
    prev_dim = 1 << (n - 1)
    dim = 1 << n  # 2^n
    table = [[None for _ in range(dim)] for _ in range(dim)]

    # Step 1: Fill in subtable 'a' (the upper left block): copy previous table
    for i in range(prev_dim):
        for j in range(prev_dim):
            table[i][j] = prev_table[i][j]

    # Step 2: Fill in subtables 'b', 'c', and 'd' by shifting indices and negating signs
    for i in range(prev_dim):
        for j in range(prev_dim):
            sign, index = prev_table[i][j]
            table[i][j + prev_dim] = (-sign, index + prev_dim)  # Block 'b'
            table[i + prev_dim][j] = (-sign, index + prev_dim)  # Block 'c'
            table[i + prev_dim][j + prev_dim] = (-sign, index + prev_dim)  # Block 'd'

    # Step 3: Adjust signs according to several rules based on overall and subtable positions.
    for i in range(dim):
        for j in range(dim):
            sign, _ = table[i][j]
            local_row = i % prev_dim  # Position within a subtable row
            local_col = j % prev_dim  # Position within a subtable column
            
            # Rule 1: First overall row or column always yield a positive sign.
            if i == 0 or j == 0:
                sign = 1

            # Rule 2: Main diagonal: 1 at (0,0) then -1 for others.
            if i == j:
                sign = 1 if i == 0 else -1

            # Determine in which subtable the (i, j) entry lies.
            in_a = (i < prev_dim and j < prev_dim)
            in_b = (i < prev_dim and j >= prev_dim)
            in_c = (i >= prev_dim and j < prev_dim)
            in_d = (i >= prev_dim and j >= prev_dim)

            # Rule 3: On a subtable’s main diagonal
            if local_row == local_col:
                if in_a or in_b:
                    # For higher n, optionally use the quaternion table for the initial indices
                    if n > 2 and local_row < 4 and local_col < 4:
                        sign = quaternion_sign(local_row, local_col)
                    elif local_row == 0:
                        sign = 1
                    else:
                        sign = -1
                elif in_c:
                    sign = 1
                elif in_d:
                    sign = -1

            # Rule 4: First column of a subtable.
            if local_col == 0:
                if in_a or in_b:
                    sign = 1
                elif in_c:
                    sign = -1 if local_row == 0 else 1
                elif in_d:
                    sign = -1 if local_row == 0 else 1

            # Rule 5: First row of a subtable.
            if local_row == 0:
                if in_a or in_b or (in_c and local_col == 0):
                    sign = 1
                elif in_d:
                    sign = -1
            if in_d and local_row == 0 and local_col > 0:
                sign = 1
            if in_d and local_col == 0 and local_row > 0:
                sign = -1

            table[i][j] = (sign, i ^ j)  # Ensure the index remains i XOR j

    return table

def multiply_elements(a, b, n):
    """
    Multiplies two elements 'a' and 'b' in a Cayley–Dickson algebra of dimension 2^n.
    
    Both 'a' and 'b' are expected to be lists (or vectors) of real coefficients 
    with length 2^n representing:
    
        a = a[0]*e0 + a[1]*e1 + ... + a[2^n - 1]*e_{2^n-1}
    
    The multiplication is performed using the Cayley–Dickson table:
    
        e_i * e_j = s_{ij} * e_{i XOR j}
    
    where s_{ij} is obtained from the Cayley–Dickson table.
    
    Returns the resulting element as a list of coefficients.
    """
    table = cayley_dickson_table(n)
    dim = 1 << n
    # Initialize the result vector with zeros.
    result = [0] * dim
    # Iterate over all pairs of basis indices.
    for i in range(dim):
        for j in range(dim):
            # Only consider nonzero contributions.
            if a[i] == 0 or b[j] == 0:
                continue
            coeff = a[i] * b[j]
            s, k = table[i][j]
            result[k] += coeff * s
    return result

def format_element(elem):
    """
    Returns a human–readable string for an element given by its list of coefficients.
    
    For example, an element [1, 2, -3, 0] (in the quaternion algebra) is returned as:
        "1 + 2e1 - 3e2"
    
    The basis element e0 (the multiplicative identity) is printed as a constant.
    """
    terms = []
    for i, coeff in enumerate(elem):
        if coeff == 0:
            continue  # Skip zero coefficients.
        # For the scalar part e0, do not print the basis symbol.
        if i == 0:
            term = f"{coeff}"
        else:
            # Format the term: omit the coefficient when it is 1 or -1 (but keep the sign).
            if coeff == 1:
                term = f"e{i}"
            elif coeff == -1:
                term = f"-e{i}"
            else:
                term = f"{coeff}e{i}"
        terms.append(term)
    if not terms:
        return "0"
    # Combine the terms with appropriate signs.
    result = terms[0]
    for term in terms[1:]:
        if term.startswith("-"):
            result += " - " + term[1:]
        else:
            result += " + " + term
    return result

# Example Usage with User Input:
if __name__ == "__main__":
    # Ask the user for the dimension parameter n (so that the algebra has dimension 2^n).
    n_input = input("Enter the parameter n (e.g., 2 for quaternions, 3 for octonions, etc.): ")
    try:
        n = int(n_input)
    except ValueError:
        print("Error: n must be an integer.")
        exit(1)
    dim = 1 << n  # Compute the dimension as 2^n.

    # Get the first element from the user.
    a_str = input(f"Enter the coefficients for element a as {dim} comma-separated numbers: ")
    try:
        a = [float(x.strip()) for x in a_str.split(",")]
    except ValueError:
        print("Error: Please enter valid numbers for the coefficients.")
        exit(1)
    if len(a) != dim:
        print(f"Error: You must provide exactly {dim} coefficients for element a.")
        exit(1)

    # Get the second element from the user.
    b_str = input(f"Enter the coefficients for element b as {dim} comma-separated numbers: ")
    try:
        b = [float(x.strip()) for x in b_str.split(",")]
    except ValueError:
        print("Error: Please enter valid numbers for the coefficients.")
        exit(1)
    if len(b) != dim:
        print(f"Error: You must provide exactly {dim} coefficients for element b.")
        exit(1)

    # Multiply the elements using our Cayley–Dickson multiplication routine.
    prod = multiply_elements(a, b, n)
    
    # Print the factors and the result in a human–readable format.
    print("\nElement a: ", format_element(a))
    print("Element b: ", format_element(b))
    print("Product  : ", format_element(prod))


In [ ]:
import re

def quaternion_sign(i, j):
    """
    Returns the sign for the quaternion multiplication e_i * e_j,
    based on a hardcoded 4x4 table. For quaternions:
    
      e0 e1 e2 e3  
    e0  1  1  1  1  
    e1  1 -1  1 -1  
    e2  1 -1 -1  1  
    e3  1  1 -1 -1  

    This is used as the base rule in the Cayley–Dickson construction.
    """
    table = [
        [1, 1, 1, 1],
        [1, -1, 1, -1],
        [1, -1, -1, 1],
        [1, 1, -1, -1]
    ]
    return table[i][j]

def cayley_dickson_table(n):
    """
    Recursively generates a multiplication table for the Cayley–Dickson algebra
    of dimension d = 2^n.
    
    Each table entry at position (i, j) is a tuple (s, k) where:
      - s is the sign (+1 or -1) determined by the recursive construction.
      - k is the index of the resulting basis element, computed as i XOR j.
      
    Base case:
      For n == 2, we generate the quaternion multiplication table.
      
    Recursive case:
      For n > 2, we build the table from the previous dimension (2^(n-1))
      by arranging four blocks (a, b, c, d) and then applying correction
      rules to adjust the signs based on the position in the overall table and
      within each subblock.
    """
    if n == 2:  # Base case: Quaternion table (4-dimensional)
        dim = 4
        table = [[None for _ in range(dim)] for _ in range(dim)]
        for i in range(dim):
            for j in range(dim):
                table[i][j] = (quaternion_sign(i, j), i ^ j)
        return table

    # Build from the previous dimension (recursive doubling)
    prev_table = cayley_dickson_table(n - 1)
    prev_dim = 1 << (n - 1)
    dim = 1 << n  # 2^n
    table = [[None for _ in range(dim)] for _ in range(dim)]

    # Step 1: Fill in subtable 'a' (the upper left block): copy previous table
    for i in range(prev_dim):
        for j in range(prev_dim):
            table[i][j] = prev_table[i][j]

    # Step 2: Fill in subtables 'b', 'c', and 'd' by shifting indices and negating signs
    for i in range(prev_dim):
        for j in range(prev_dim):
            sign, index = prev_table[i][j]
            table[i][j + prev_dim] = (-sign, index + prev_dim)  # Block 'b'
            table[i + prev_dim][j] = (-sign, index + prev_dim)  # Block 'c'
            table[i + prev_dim][j + prev_dim] = (-sign, index + prev_dim)  # Block 'd'

    # Step 3: Adjust signs according to several rules based on overall and subtable positions.
    for i in range(dim):
        for j in range(dim):
            sign, _ = table[i][j]
            local_row = i % prev_dim  # Position within a subtable row
            local_col = j % prev_dim  # Position within a subtable column
            
            # Rule 1: First overall row or column always yield a positive sign.
            if i == 0 or j == 0:
                sign = 1

            # Rule 2: Main diagonal: 1 at (0,0) then -1 for others.
            if i == j:
                sign = 1 if i == 0 else -1

            # Determine in which subtable the (i, j) entry lies.
            in_a = (i < prev_dim and j < prev_dim)
            in_b = (i < prev_dim and j >= prev_dim)
            in_c = (i >= prev_dim and j < prev_dim)
            in_d = (i >= prev_dim and j >= prev_dim)

            # Rule 3: On a subtable’s main diagonal
            if local_row == local_col:
                if in_a or in_b:
                    # For higher n, optionally use the quaternion table for the initial indices
                    if n > 2 and local_row < 4 and local_col < 4:
                        sign = quaternion_sign(local_row, local_col)
                    elif local_row == 0:
                        sign = 1
                    else:
                        sign = -1
                elif in_c:
                    sign = 1
                elif in_d:
                    sign = -1

            # Rule 4: First column of a subtable.
            if local_col == 0:
                if in_a or in_b:
                    sign = 1
                elif in_c:
                    sign = -1 if local_row == 0 else 1
                elif in_d:
                    sign = -1 if local_row == 0 else 1

            # Rule 5: First row of a subtable.
            if local_row == 0:
                if in_a or in_b or (in_c and local_col == 0):
                    sign = 1
                elif in_d:
                    sign = -1
            if in_d and local_row == 0 and local_col > 0:
                sign = 1
            if in_d and local_col == 0 and local_row > 0:
                sign = -1

            table[i][j] = (sign, i ^ j)  # Ensure the index remains i XOR j

    return table

def multiply_elements(a, b, n):
    """
    Multiplies two elements 'a' and 'b' in a Cayley–Dickson algebra of dimension 2^n.
    
    Both 'a' and 'b' are expected to be lists (or vectors) of real coefficients 
    with length 2^n representing:
    
        a = a[0]*e0 + a[1]*e1 + ... + a[2^n - 1]*e_{2^n-1}
    
    The multiplication is performed using the Cayley–Dickson table:
    
        e_i * e_j = s_{ij} * e_{i XOR j}
    
    where s_{ij} is obtained from the Cayley–Dickson table.
    
    Returns the resulting element as a list of coefficients.
    """
    table = cayley_dickson_table(n)
    dim = 1 << n
    # Initialize the result vector with zeros.
    result = [0] * dim
    # Iterate over all pairs of basis indices.
    for i in range(dim):
        for j in range(dim):
            # Only consider nonzero contributions.
            if a[i] == 0 or b[j] == 0:
                continue
            coeff = a[i] * b[j]
            s, k = table[i][j]
            result[k] += coeff * s
    return result

def format_element(elem):
    """
    Returns a human–readable string for an element given by its list of coefficients.
    
    For example, an element [1, 2, -3, 0] (in the quaternion algebra) is returned as:
        "1 + 2e1 - 3e2"
    
    The basis element e0 (the multiplicative identity) is printed as a constant.
    """
    terms = []
    for i, coeff in enumerate(elem):
        if coeff == 0:
            continue  # Skip zero coefficients.
        # For the scalar part e0, do not print the basis symbol.
        if i == 0:
            term = f"{coeff}"
        else:
            # Format the term: omit the coefficient when it is 1 or -1 (but keep the sign).
            if coeff == 1:
                term = f"e{i}"
            elif coeff == -1:
                term = f"-e{i}"
            else:
                term = f"{coeff}e{i}"
        terms.append(term)
    if not terms:
        return "0"
    # Combine the terms with appropriate signs.
    result = terms[0]
    for term in terms[1:]:
        if term.startswith("-"):
            result += " - " + term[1:]
        else:
            result += " + " + term
    return result

def parse_element(s, dim):
    """
    Parses a hypercomplex element provided as a string (e.g., "1 + 2e1 - 3e2" or "e2")
    and returns a list of coefficients of length 'dim' corresponding to:
    
        a = a[0]*e0 + a[1]*e1 + ... + a[dim-1]*e_{dim-1}
    
    The parser assumes that if no coefficient is explicitly given before a basis element,
    the coefficient is 1 (or -1 if a minus sign is present). Scalar terms (without an 'e')
    are interpreted as the coefficient for e0.
    """
    # Remove spaces for easier processing.
    s = s.replace(" ", "")
    if not s:
        raise ValueError("Empty input for hypercomplex element.")
    # Ensure the string starts with a sign.
    if s[0] not in "+-":
        s = "+" + s
    # Split the string into tokens where each token starts with a sign.
    tokens = re.findall(r'[+-][^+-]+', s)
    
    # Initialize coefficients for each basis element.
    coeffs = [0.0] * dim
    for token in tokens:
        if 'e' in token:
            # Split at the first occurrence of 'e'.
            parts = token.split('e', 1)
            coeff_str = parts[0]
            index_str = parts[1]
            # If no explicit coefficient is given, interpret as 1 or -1.
            if coeff_str in ("+", "-"):
                coeff = 1.0 if coeff_str == "+" else -1.0
            else:
                try:
                    coeff = float(coeff_str)
                except ValueError:
                    raise ValueError(f"Invalid coefficient in token: {token}")
            try:
                index = int(index_str)
            except ValueError:
                raise ValueError(f"Invalid basis index in token: {token}")
        else:
            # No basis element indicated; this is the scalar term (basis e0).
            try:
                coeff = float(token)
            except ValueError:
                raise ValueError(f"Invalid scalar term: {token}")
            index = 0

        if index < 0 or index >= dim:
            raise ValueError(f"Basis index {index} out of range for dimension {dim}.")
        coeffs[index] += coeff
    return coeffs

# Main program using user inputs:
if __name__ == "__main__":
    print("Hypercomplex Element Multiplication via Cayley–Dickson Construction")
    
    # Get the parameter n from the user.
    n_input = input("Enter n (n=1 for Complex, n=2 for Quaternions, n=3 for Octonions, etc.): ")
    try:
        n = int(n_input)
    except ValueError:
        print("Error: n must be an integer.")
        exit(1)
    
    # Calculate the dimension.
    dim = 1 << n  # 2^n
    print(f"\nThis algebra has dimension {dim} (basis elements e0, e1, ..., e{dim-1}).\n")
    
    # Get the first element from the user.
    a_str = input("Enter the first element (e.g., '1 + 2e1 - 3e2'):\n")
    try:
        a = parse_element(a_str, dim)
    except ValueError as e:
        print("Error parsing first element:", e)
        exit(1)
    
    # Get the second element from the user.
    b_str = input("\nEnter the second element (e.g., '4 - e1 + 0.5e3'):\n")
    try:
        b = parse_element(b_str, dim)
    except ValueError as e:
        print("Error parsing second element:", e)
        exit(1)
    
    # Multiply the elements using the Cayley–Dickson multiplication routine.
    prod = multiply_elements(a, b, n)
    
    # Print the result.
    print("\nResult of multiplication:")
    print(a_str,"*",b_str,"=",format_element(prod))
    table = cayley_dickson_table(n)
    print(f"Cayley-Dickson Multiplication Table (n={n}, dimension={2**n}):")
    for row in table:
        for sign_val, index in row:
            print(f"{'+' if sign_val == 1 else '-' }e{index}", end="\t")
        print()
    print("-" * 40)
